### Home Network Assistant
---

In this notebook, we will create our first crew agent. This crew agent is the Home Network Agent. This agent will have access to a knowledge base for retrieval at runtime which will contain API specs on the home networking system. In this notebook, we will create a Knowledge base for our Home Network Agent. This agent will have access to a knowledge base for retrieval at runtime which will contain API specs on the home networking system. 

CrewAI enables you to create AI teams where each agent has specific roles, tools, and goals, working together to accomplish complex tasks.

Think of it as assembling your dream team - each member (agent) brings unique skills and expertise, collaborating seamlessly to achieve your objectives. For more information on CrewAI, view [here](https://docs.crewai.com/introduction)

**Agent code generation and execution workflow**:

1. The workflow starts with information retrieval. We create a knowledge base, and store the information from the `Home Network openAPI spec` into the knowledge base. The OpenAPI spec is provided by the user in the `data` folder.

1. This knowledge base will be wrapped within a lambda function that will be invoked based on the user query. It will `retrieve` the top `k` results from the knowledge base and send it as an input to the next step.

1. Next, a `Home Networking` assistant agent will have access to the required API specs to use to generate code. It will use the information from the Knowledge base and generate code for the given API spec, save the code and execute the code based on the parameters provided by the user.

### Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

#### Restart kernel

If you face issues to apply the latest multi-agent capabilities, uncomment this line to restart kernel to ensure packages updates to take effect

In [1]:
# Install crew ai. For installation steps, follow the instructions here: https://docs.crewai.com/installation
!pip install 'crewai[tools]'

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import IPython

# IPython.Application.instance().kernel.do_shutdown(True)

In [3]:
# Check your boto3 version
!pip freeze | grep boto3

boto3==1.36.10


In [4]:
# import the required packages and libraries
import os
import sys
import boto3
import logging
from typing import Optional
from dotenv import load_dotenv
# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))
# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
# Add the parent directory to sys.path
sys.path.append(parent_dir)
from globals import *

/Users/madhurpt/multi-agent-code-gen-and-execution


In [5]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
# configure the sts client, the boto3 session and other variables
sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = "us-east-1" if session.region_name is None else session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

[2025-02-05 17:20:19,900] p32670 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [7]:
import sys
sys.path.insert(0, ".")
sys.path.insert(1, "..")
# Import utility functions and helper functions for agents
from utils.utils import *

### Load the config file
--- 

Load the config file that contains information on the models, data directories, etc.

In [8]:
# Get the absolute path to the config file
# This config file contains data about the directory paths, the API specs that
# are used to generate the code, and the agent foundation models that are used to generate the code.
BASE_DIR = os.path.abspath(sys.path[1])
CONFIG_FPATH = os.path.join(BASE_DIR, CONFIG_FNAME)
config_data = load_config(CONFIG_FPATH)
logger.info(f"Loaded config from local file system: {json.dumps(config_data, indent=2)}")

[2025-02-05 17:20:22,347] p32670 {utils.py:61} INFO - Loading config from local file system: /Users/madhurpt/multi-agent-code-gen-and-execution/config.yaml
[2025-02-05 17:20:22,354] p32670 {utils.py:64} INFO - Loaded config from local file system: {'general': {'name': 'doorbell-home-network-agentic-system', 'description': 'This agentic system shows multi-agent collaboration a home network system and a doorbell system.'}, 'dir_paths': {'data_prefix': 'data', 'prompts_prefix': 'prompt_templates', 'results_prefix': 'results', 'agent_instructions_prefix': 'agent_instructions', 'agent_instructions': {'doorbell_agent_instructions': 'doorbell_agent_instructions.txt', 'home_network_agent_instructions': 'home_network_agent_instructions.txt'}, 'code_gen_prompts_prefix': 'code_gen_prompts', 'code_gen_prompts': {'home_network_code_generation_prompt': 'home_network_code_generation_prompt.txt', 'doorbell_code_generation_prompt': 'doorbell_code_generation_prompt.txt'}, 'api_specs': {'home_network_api

## Code Generation Prompt: Bedrock prompt management
---

Let's create our sample code generation prompt by leveraging on Prompt Management for Amazon Bedrock. This is the prompt that is used by the sub agent to generate the code when it calls the generate_code tool.

In [9]:
def read_prompt_from_file(file_path: str) -> str:
    with open(file_path, 'r') as file:
        return file.read().strip()

prompt_file_path = os.path.join(
    config_data['dir_paths']['code_gen_prompts_prefix'],
    config_data['dir_paths']['code_gen_prompts'].get('home_network_code_generation_prompt')
)

absolute_prompt_fpath = os.path.join(
    parent_dir,
    prompt_file_path
)

prompt_template_code_gen: str = read_prompt_from_file(absolute_prompt_fpath)
print(f"Code generation prompt that will be saved in prompt management within Bedrock: {prompt_template_code_gen}")

Code generation prompt that will be saved in prompt management within Bedrock: With the content that you have access to, use that to generate executable python code to call the required API based on the user query and save it locally.

Your primary function is to assist users by interpreting their requests, analyzing the knowledge base content, using the required parameters in the input JSON string to generate appropriate Python code to call the home network API endpoints.

In your main function, NEVER add parameters as placeholders. Always use the parameters provided in the input JSON string and use those in the code that you generate. If there are parameters that are missing from the input JSON string
that are required to generate fully executable code, then do not generate the code and ask the user for the required parameters.

Here are your key attributes and instructions:

1. Knowledge Base Response Understanding:
- You have access to results from the knowledge base that contains 

In [10]:
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = region)
response = bedrock_agent.create_prompt(
    name = f"prompt-for-home-network-code-gen",
    description = "Code generation prompt template that is used by the home networking agent to generate code",
    variants = [
        {
            "name": "variantOne",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "input"
                        },
                        {
                            "name": "output"
                        }
                    ],
                    "text": prompt_template_code_gen
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = "variantOne"
)

print(json.dumps(response, indent=2, default=str))
promptId = response["id"]
promptArn = response["arn"]
promptName = response["name"]
print(f"Prompt ID: {promptId}\nPrompt ARN: {promptArn}\nPrompt Name: {promptName}")

{
  "ResponseMetadata": {
    "RequestId": "d7c7c49b-ca28-42de-81aa-9554d935ac69",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Wed, 05 Feb 2025 22:20:22 GMT",
      "content-type": "application/json",
      "content-length": "6117",
      "connection": "keep-alive",
      "x-amzn-requestid": "d7c7c49b-ca28-42de-81aa-9554d935ac69",
      "x-amz-apigw-id": "FiCWFFMhoAMEGwg=",
      "x-amzn-trace-id": "Root=1-67a3e426-7b37dc344812b6043b8c0d6e"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:218208277580:prompt/8FV0HVCA3A",
  "createdAt": "2025-02-05 22:20:22.577240+00:00",
  "defaultVariant": "variantOne",
  "description": "Code generation prompt template that is used by the home networking agent to generate code",
  "id": "8FV0HVCA3A",
  "name": "prompt-for-home-network-code-gen",
  "updatedAt": "2025-02-05 22:20:22.577240+00:00",
  "variants": [
    {
      "name": "variantOne",
      "templateConfiguration": {
        "text": {
          "input

In [11]:
# Now that we have a draft prompt, we can create a version from it.
response = bedrock_agent.create_prompt_version(
    promptIdentifier = promptId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "4769cedc-3a7a-47be-8591-58b94283b78a",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Wed, 05 Feb 2025 22:20:23 GMT",
      "content-type": "application/json",
      "content-length": "6008",
      "connection": "keep-alive",
      "x-amzn-requestid": "4769cedc-3a7a-47be-8591-58b94283b78a",
      "x-amz-apigw-id": "FiCWNGpyoAMEkCQ=",
      "x-amzn-trace-id": "Root=1-67a3e427-27778bb97840fc8d0b95a3a2"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:218208277580:prompt/8FV0HVCA3A:1",
  "createdAt": "2025-02-05 22:20:23.439099+00:00",
  "defaultVariant": "variantOne",
  "id": "8FV0HVCA3A",
  "name": "prompt-for-home-network-code-gen",
  "updatedAt": "2025-02-05 22:20:23.439099+00:00",
  "variants": [
    {
      "name": "variantOne",
      "templateConfiguration": {
        "text": {
          "inputVariables": [
            {
              "name": "input"
            },
            {
              "name": 

## JSON Knowledge source
---

In this section of the solution we will be creating our Crew Agent, initialize the instruction for the home networking system, and then provide it with a `knowledge source`. Knowledge in CrewAI is a powerful system that allows AI agents to access and utilize external information sources during their tasks. Think of it as giving your agents a reference library they can consult while working. 

We will be creating a `JSON Knowledge Source` that will store embeddings from the `JSON` API specs that we will provide.

In [12]:
api_specs: Dict = config_data['dir_paths'].get('api_specs')
logger.info(f"API specs that are provided are as follows: {api_specs}")

[2025-02-05 17:20:24,623] p32670 {2846991394.py:2} INFO - API specs that are provided are as follows: {'home_network_api_spec': 'home_network_openapi_spec.json', 'doorbell_api_spec': 'doorbell_openapi_spec.json'}


In [13]:
import os
import shutil
from crewai.knowledge.source.json_knowledge_source import JSONKnowledgeSource

if not os.path.exists(config_data['dir_paths']['data_prefix']):
    os.makedirs(config_data['dir_paths']['data_prefix'])

# initialize an embedder
embedder = {
    "provider": "bedrock",
    "config": {
        "model": config_data['model_information']['embedding_model']
    },
}

try:
    if os.path.exists(config_data['dir_paths']['data_prefix']):
        # Create knowledge directory if it doesn't exist
        os.makedirs('knowledge', exist_ok=True)
        source_path = os.path.join('..', config_data['dir_paths']['data_prefix'], 
                                 api_specs.get('home_network_api_spec'))
        destination_path = api_specs.get('home_network_api_spec')  
        logger.info(f"Source path: {os.path.abspath(source_path)}")
        logger.info(f"Destination path: {os.path.abspath(destination_path)}")
        shutil.copy(source_path, destination_path)
        api_spec = JSONKnowledgeSource(file_paths=destination_path, embedder=embedder)
except Exception as e:
    logger.error(f"Error creating knowledge source: {e}")
    logger.info(f"Current working directory: {os.getcwd()}")
    logger.info(f"Directory contents: {os.listdir('.')}")


[2025-02-05 17:20:26,646] p32670 {_client.py:1026} INFO - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
[2025-02-05 17:20:27,829] p32670 {3354197254.py:23} INFO - Source path: /Users/madhurpt/multi-agent-code-gen-and-execution/data/home_network_openapi_spec.json
[2025-02-05 17:20:27,830] p32670 {3354197254.py:24} INFO - Destination path: /Users/madhurpt/multi-agent-code-gen-and-execution/0_home_network_assistant/home_network_openapi_spec.json


In [14]:
logger.info(f"Loaded the API spec. API spec information: {api_spec}")

[2025-02-05 17:20:27,837] p32670 {2981485105.py:1} INFO - Loaded the API spec. API spec information: chunk_size=4000 chunk_overlap=200 chunks=[] chunk_embeddings=[] storage=None metadata={} collection_name=None file_path=None file_paths='home_network_openapi_spec.json' content={PosixPath('knowledge/home_network_openapi_spec.json'): 'openapi: 3.0.3\ninfo:   title: SmartHome Security API\n  description: API specification for managing smart cameras and doorbells\n  version: 1.0.0\n  contact:     name: SmartHome Security Support\n    email: support@smarthomesecurity.example.com\n\n\nservers:   -     url: https://api.smarthomesecurity.example.com/v1\n    description: Production server\n\n  -     url: https://api-staging.smarthomesecurity.example.com/v1\n    description: Staging server\n\n\ntags:   -     name: cameras\n    description: Camera management and control\n\n  -     name: firmware\n    description: Firmware management operations\n\n  -     name: metrics\n    description: Device met

### Define tools that the CrewAI agent will have access to
---

In this portion of the notebook, we create different tools that the CrewAI agent will have access to. This includes a tool to query from the knowledge base, generate code based on the user query and the API to call, save the code and finally execute the code to provide the final result to the user.

In [15]:
# Load the environment variables that are defined in the ".env" file.
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [16]:
import os
import sys
import ast
import json
import time
import boto3
import random
import logging
import tempfile
import subprocess
from crewai.tools import tool
from crewai import LLM, Agent, Task, Crew

# Information on the code generation model
code_gen_data = config_data['code_generation_model_information']

def get_prompt_template(prompt_id: str) -> str:
    """
    Retrieve a prompt template from Bedrock prompt management.
    """
    try:
        client = boto3.client("bedrock-agent", region_name=os.environ.get("REGION", "us-east-1"))
        result = client.get_prompt(promptIdentifier=prompt_id)
        template = result["variants"][0]["templateConfiguration"]["text"]["text"]
        return template
    except Exception as e:
        logger.error(f"Error getting prompt template: {e}")
        raise

def call_bedrock(model_id: str, messages: list, temp: float, max_tokens: int, top_p: float) -> str:
    """
    Invoke Bedrock's converse API with the provided parameters.
    """
    try:
        client = boto3.client("bedrock-runtime")
        inference_config = {
            "temperature": temp,
            "maxTokens": max_tokens,
            "topP": top_p
        }
        start = time.time()
        response = client.converse(
            modelId=model_id,
            messages=messages,
            system=[{"text": "You are an expert in generating Python code for home networking APIs."}],
            inferenceConfig=inference_config
        )
        elapsed = time.time() - start
        logger.info(f"Bedrock response in {elapsed:.2f} sec")
        # Extract the generated code.
        code = response["output"]["message"]["content"][0]["text"]
        return code
    except Exception as e:
        logger.error(f"Error calling Bedrock converse: {e}")
        raise


# ─── TOOL 1: CODE GENERATION VIA BEDROCK ──────────────────────────────────────
@tool("bedrock_code_generation_tool")
def bedrock_code_generation_tool(query: str) -> str:
    """
    This tool generates Python code using Bedrock. This is the first tool that is called as a part
    of the crew, to check for the knowledge base content and then check if the user has provided all of the
    parameters or not. If all parameters are provided, this tool is used to generate the code.
    """
    try:
        prompt_id = promptId
        template = get_prompt_template(prompt_id)
        # Fill the prompt template.
        formatted_prompt = template.format(
            user_query=query,
            auth_token=os.getenv("HOME_NETWORK_AUTH_TOKEN")
        )
        messages = [{"role": "user", "content": formatted_prompt}]
        # Get Bedrock model and inference parameters from environment variables.
        model = code_gen_data.get('code_generation_model')
        temperature = code_gen_data.get('temperature', 0.1)
        top_p = code_gen_data.get('top_p', 0.9)
        max_tokens = code_gen_data.get('max_tokens', 4096)
        code_output = call_bedrock(model, messages, temperature, max_tokens, top_p)
        return code_output
    except Exception as err:
        logger.error(f"bedrock_code_generation_tool error: {err}")
        return f"Error generating code: {err}"


# ─── TOOL 2: SAVE GENERATED CODE ─────────────────────────────────────────────
@tool("save_code_tool")
def save_code_tool(code_content: str) -> str:
    """
    Saves the provided code content to a temporary file and returns the file path.
    """
    try:
        temp_dir = tempfile.mkdtemp(prefix="code_exec_")
        file_path = os.path.join(temp_dir, "generated_code.py")
        with open(file_path, "w") as fp:
            fp.write(code_content)
        logger.info(f"Code saved to {file_path}")
        return file_path
    except Exception as e:
        logger.error(f"Error in save_code_tool: {e}")
        return f"Error saving code: {e}"


# ─── TOOL 3: EXECUTE GENERATED CODE ───────────────────────────────────────────
@tool("execute_code_tool")
def execute_code_tool(file_path: str) -> dict:
    """
    Executes the code saved at file_path using the current Python interpreter.
    Returns a dictionary with stdout, stderr, return code, and success flag.
    """
    try:
        # For debugging, log the code that is about to be executed.
        with open(file_path, "r") as fp:
            code_text = fp.read()
        logger.info(f"Executing code:\n{code_text}")

        result = subprocess.run(
            [sys.executable, file_path],
            capture_output=True,
            text=True,
            timeout=int(os.environ.get("CODE_EXECUTION_TIMEOUT", "30")),
            env=os.environ.copy()
        )
        exec_result = {
            "stdout": result.stdout,
            "stderr": result.stderr,
            "return_code": result.returncode,
            "success": (result.returncode == 0)
        }
        logger.info(f"Execution result: {exec_result}")
        return exec_result
    except subprocess.TimeoutExpired:
        logger.error("Execution timed out")
        return {
            "stdout": "",
            "stderr": "Execution timed out",
            "return_code": -1,
            "success": False
        }
    except Exception as err:
        logger.error(f"Error executing code: {err}")
        return {
            "stdout": "",
            "stderr": str(err),
            "return_code": -1,
            "success": False
        }

### Create the home networking agent and Crew
---

In this portion of the solution, we will create an agent for home networking configuration assistance, that will have access to the tools defined above. We will then add this agent to the crew with the home networking knowledge base, which contains information about the API spec.

In [17]:
instructions_path: str = os.path.join(config_data['dir_paths']['agent_instructions_prefix'], 
                                      config_data['dir_paths']['agent_instructions'].get('home_network_agent_instructions'))
agent_instruction = open(os.path.join(parent_dir, instructions_path), 'r').read()
agent_instruction

"Human: You're a home networking management assistant that is able to help with different tasks across different home networking configurations. \nYou are provided with functions that you need to answer the user question. The functions that you have access to are in the <tools></tools> tags:\n\n<functions>\n- bedrock_code_generation_tool\n- save_code_tool\n- execute_code_tool\n</functions>\n\nFor the given query '{query}', follow the steps below:\n\nFollow the steps below in the <steps></steps> xml tags in the given order when a user asks a new question:\n\n<steps>\n1. STEP 1: First, a user will provide you with a query. Use the home network knowledge base to retrieve information about that query if you are encountering the query for the first time.\nOnce you retrieve information from the home network knowledge base, check if all parameters are provided by the user. If the user has not provided the authorization token, that is fine, do not ask for that because that is private informati

In [18]:
from crewai.tools import tool
from crewai import LLM, Agent, Task, Crew

# This is the FM used for the home networking sub agent
llm_instance = LLM(
    model=config_data['model_information']['home_network_sub_agent_model'],
    temperature=0.1,
    max_tokens=2048,
    top_p=0.9,
)

# Create the home networking sub agent along with the tools assigned to it
home_network_agent = Agent(
    role="Home Networking Configuration Assistant",
    goal=(
        agent_instruction
    ),
    backstory=(
        "I am an expert agent who combines KB queries with advanced code generation and execution capabilities to streamline home networking tasks."
    ),
    tools=[bedrock_code_generation_tool, save_code_tool, execute_code_tool],
    verbose=True,
    llm=llm_instance,
    knowledge_source=[api_spec]
)

# Define a task that explains the complete flow.
code_generation_and_execution_task = Task(
    description=(
        "For the given query '{query}', follow the steps to refer to the knowledge, then generate the code, save it and execute the code.,"
    ),
    expected_output="The output produced by the executed Python code.",
    agent=home_network_agent,
    output_file="home_network_generated_code.py"
)

In [19]:
# Finally, create a crew that holds the agent and its task.
crew = Crew(
    agents=[home_network_agent],
    tasks=[code_generation_and_execution_task],
    verbose=True,
)

In [20]:
crew_inputs = {
    'query': 'What is the signal strength of my porch camera?'
}

crew.kickoff(inputs=crew_inputs) 

17:20:31 - LiteLLM:INFO: utils.py:2825 - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
[2025-02-05 17:20:31,665] p32670 {utils.py:2825} INFO - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
[2025-02-05 17:20:31,678] p32670 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


# Agent: Home Networking Configuration Assistant
## Task: For the given query 'What is the signal strength of my porch camera?', follow the steps to refer to the knowledge, then generate the code, save it and execute the code.,


[2025-02-05 17:20:33,663] p32670 {_client.py:1026} INFO - HTTP Request: POST https://bedrock-runtime.us-west-2.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/converse "HTTP/1.1 200 OK"
17:20:33 - LiteLLM:INFO: utils.py:1030 - Wrapper: Completed Call, calling success_handler
[2025-02-05 17:20:33,665] p32670 {utils.py:1030} INFO - Wrapper: Completed Call, calling success_handler
[2025-02-05 17:20:33,848] p32670 {3890103828.py:54} ERROR - Error calling Bedrock converse: Parameter validation failed:
Invalid type for parameter messages[0].content, value: With the content that you have access to, use that to generate executable python code to call the required API based on the user query and save it locally.

Your primary function is to assist users by interpreting their requests, analyzing the knowledge base content, using the required parameters in the input JSON string to generate appropriate Python code to call the home network API endpoints.

In your main function, NEVER ad



# Agent: Home Networking Configuration Assistant
## Using tool: bedrock_code_generation_tool
## Tool Input: 
"{\"query\": \"What is the signal strength of my porch camera?\"}"
## Tool Output: 
Error generating code: Parameter validation failed:
Invalid type for parameter messages[0].content, value: With the content that you have access to, use that to generate executable python code to call the required API based on the user query and save it locally.

Your primary function is to assist users by interpreting their requests, analyzing the knowledge base content, using the required parameters in the input JSON string to generate appropriate Python code to call the home network API endpoints.

In your main function, NEVER add parameters as placeholders. Always use the parameters provided in the input JSON string and use those in the code that you generate. If there are parameters that are missing from the input JSON string
that are required to generate fully executable code, then do not

[2025-02-05 17:20:37,411] p32670 {_client.py:1026} INFO - HTTP Request: POST https://bedrock-runtime.us-west-2.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/converse "HTTP/1.1 200 OK"
17:20:37 - LiteLLM:INFO: utils.py:1030 - Wrapper: Completed Call, calling success_handler
[2025-02-05 17:20:37,412] p32670 {utils.py:1030} INFO - Wrapper: Completed Call, calling success_handler
17:20:37 - LiteLLM:INFO: utils.py:2825 - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
[2025-02-05 17:20:37,417] p32670 {utils.py:2825} INFO - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock


 

Action 'None' don't exist, these are the only available Actions:
Tool Name: bedrock_code_generation_tool
Tool Arguments: {'query': {'description': None, 'type': 'str'}}
Tool Description: 
    This tool generates Python code using Bedrock. This is the first tool that is called as a part
    of the crew, to check for the knowledge base content and then check if the user has provided all of the
    parameters or not. If all parameters are provided, this tool is used to generate the code.
    
Tool Name: save_code_tool
Tool Arguments: {'code_content': {'description': None, 'type': 'str'}}
Tool Description: 
    Saves the provided code content to a temporary file and returns the file path.
    
Tool Name: execute_code_tool
Tool Arguments: {'file_path': {'description': None, 'type': 'str'}}
Tool Description: 
    Executes the code saved at file_path using the current Python interpreter.
    Returns a dictionary with stdout, stderr, return code, and success flag.
    



# Agent: Home Netw

[2025-02-05 17:20:40,554] p32670 {_client.py:1026} INFO - HTTP Request: POST https://bedrock-runtime.us-west-2.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/converse "HTTP/1.1 200 OK"
17:20:40 - LiteLLM:INFO: utils.py:1030 - Wrapper: Completed Call, calling success_handler
[2025-02-05 17:20:40,555] p32670 {utils.py:1030} INFO - Wrapper: Completed Call, calling success_handler
17:20:40 - LiteLLM:INFO: utils.py:2825 - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
[2025-02-05 17:20:40,665] p32670 {utils.py:2825} INFO - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock




# Agent: Home Networking Configuration Assistant
## Using tool: bedrock_code_generation_tool
## Tool Input: 
"{\"query\": \"What is the signal strength of my porch camera?\"}"
## Tool Output: 
I tried reusing the same input, I must stop using this action input. I'll try something else instead.




KeyboardInterrupt: 

[2025-02-05 17:21:05,873] p32670 {__init__.py:362} ERROR - Exception while exporting Span batch.
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py", line